In [102]:
# starts with ONE dimension
### sets all y-values to 0
# neglects the problem of how to divide a 2D/3D-area-cells in each rank

### initial example
# 2 ranks
# 4 cells

#pseudo code:
* For hver partikkel, sjekk hvilken rank som eier den cellen den er i.
  * De som fortsatt er i en celle som er eid av ranken som har partikkelen trenger ikke sendes.
  * De som er i en celle eid av en annen rank må sendes.

* Hver rank kan nå regne ut hvor mange partikler den skal sende til alle andre ranks, men den kan ikke selv regne ut hvor mange den skal motta. Denne informasjonen er nødvendig for å kunne allokere plass til å ta imot.

* For å utveksle denne informasjonen har jeg følgt følgende strategi (forsøkt illustrert i det andre vedlagte bildet)
  * På hver rank, sett opp et kvadratisk array (datatype integer), med størrelse antall ranks x antall ranks.
  * Sett alle elementer til 0
  * La rader representere antall partikler som skal sendes, og kolonner partikler som skal mottas
  * (Merk at denne arrayen som regel ikke vil være symmetrisk om diagonalen)
  * Så for eksempel rad 2 forteller hvor mange partikler som skal sendes fra rank 2, til hver av de andre
  * På dette tidspunktet har hver rank nok informasjon til å fylle ut sin egen rad.
  * Kall deretter allreduce med sum-operatoren.
  * Alle ranks har nå en komplett kopi av hele arrayet, og vet dermed hvor mange partikler de skal ta imot.

* Nå er man klar for å begynne selve utvekslingen av partikler.
  * For hver rank, sett opp et array for å holde partikler som skal sendes
    * Det finnes minst to måter å gjøre dette:
      * Den ene er å sette opp et array av størrelse antall ranks x max antall partikler som skal sendes
      * Den andre er å lage en liste av arrays, der hver array holder partikler som skal sendes til en gitt rank, og har så mange elementer som skal sendes dit.
      * Fordelen med den første metoden er at den er enkel
      * Ulempen med den første metoden er at den potensielt kan gi veldig stor minne-overhead
  * Gjør tilsvarende for partikler som skal mottas
  * På hver rank, sett opp en non-blocking receive (MPI_irecv i fortran/c) for å ta imot fra alle andre ranks
  * På hver rank, sett opp en non-blocking send (MPI_isend) for å sende til alle andre ranks
  * På hver rank, kall mpi_waitall med referanse til alle send og receive kall. Denne gjør at man ikke går videre før all kommunikasjon er avsluttet.


In [ ]:
### TODO:implement  exception handling? primary "out of bounds" and wrong data type

In [112]:
### MPI in script

# example to run:
#mpiexec -n 2 python NAME.py
# where 2 is the number of processes

import numpy as np
from mpi4py import MPI
#from matplotlib import pyplot as plt
#plt.style.use('bmh')

#comm = MPI.COMM_WORLD
#rank = comm.Get_rank()
#my_mpi_size = comm.Get_size()

In [94]:
#######
# for testing in jupyter notebook:
rank = 2

In [123]:
# initializing spatial properties
x_start = 0
x_end = 1
y_start = 0
y_end = 0 # 1D example

x_len = x_end - x_start
y_len = y_end - y_start

In [96]:
# initializing "processes/rank" properties
# TODO: implement "find biggest factor"-function or another dynamical function to determine numer of cells in each direction

# number of cells in each direction (0 in z direction at first draft/prototype)
cells_dir_n = 4

cell_x_n = cells_dir_n

#cell_y_n = cells_dir_n
#cell_n = cell_x_n*cell_y_n
# for 1D example/prototype:
cell_y_n = 0
cell_n = cell_x_n

In [97]:
# Particle "object" is defined in several arrays
# TODO: define a "particle-class/tuple"?
# particle_id is bound to the index in the x/y-arrays

# total number of particles
particle_n = 10
particle_x = np.linspace(x_start, x_end, particle_n, endpoint=False)
particle_y = np.linspace(y_start, y_end, particle_n, endpoint=False)

In [160]:
# function to find which process/rank of cell
### this function determines how the cells are distributed to ranks
### discussion: how to do this distribution
def find_rank_from_cell(cell_id):
    return cell_id % rank

# function to find which cell of position
### this function determines how the cells are distributed geometric
### discussion: how to do this distribution
def find_cell_from_position(x):
    return np.floor(((x - x_start)/(x_len))*cell_x_n) # for 1D

# function to find which position of particle
# in 2D, return a rank 2 array?
def find_pos_from_particle(particle_id):
    return particle_x[particle_id] # for 1D
    #return (particle_x[particle_id], particle_y[particle_id])

In [159]:
# for debugging:
#find_cell_rank(find_pos_cell(find_particle_pos(8)))


1.0

0.80000000000000004

TypeError: find_pos_cell() missing 1 required positional argument: 'y'

In [4]:

# Figure out which particles need to be moved
# Each proc. fills the rows of communicationarray with what needs to be sent
communicationarray = 0
# Reset arrays telling which particles are to be sent
particles % send    = .false.
particles % send_to = -1

In [ ]:
# Only set up communicationarray if my_mpi_size > 1
print('my_mpi_size = ', my_mpi_size)
if (my_mpi_size > 1):
    # Only do work if this rank has particles
    print()'Preparing communcationarray)
    if (particles % nparticles > 0)